# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [0]:
from scipy import stats
dir(stats)
#yes!

In [15]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [16]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [17]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test. We will also dig a bit deeper into how the test statistic is calculated based on the sample error, and visually what it looks like to have 1 or 2 "tailed" t-tests.

In [0]:
# TODO - during class, but please help!

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [19]:
import pandas as pd 

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',header=None)

df.shape
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [20]:

rename= {0:'party',1:'handicap',2:'water_cost_share',
         3:'budget_resolution',4:'physician_fee_freeze',
        5:'el_savador_aid',6:'religious_groups_schools',
        7:'anti_satellite_test_ban',8:'nicaraguan_aid',
        9:'mx_missile',10:'immigration',11:'synfuels_cutback',
        12:'education_spending',13:'superfund_lawsuit',
        14:'crime',15:'duty_free_exports',16:'s_africa_export'}

df = df.rename(index=str, columns=rename)
df.head()

,party,handicap,water_cost_share,budget_resolution,physician_fee_freeze,el_savador_aid,religious_groups_schools,anti_satellite_test_ban,nicaraguan_aid,mx_missile,immigration,synfuels_cutback,education_spending,superfund_lawsuit,crime,duty_free_exports,s_africa_export
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [24]:
#df = df.set_index('party')
df.head()

,handicap,water_cost_share,budget_resolution,physician_fee_freeze,el_savador_aid,religious_groups_schools,anti_satellite_test_ban,nicaraguan_aid,mx_missile,immigration,synfuels_cutback,education_spending,superfund_lawsuit,crime,duty_free_exports,s_africa_export
party,,,,,,,,,,,,,,,,
republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [26]:
df = df.replace(['n','y'],[0,1])
df.head()

,handicap,water_cost_share,budget_resolution,physician_fee_freeze,el_savador_aid,religious_groups_schools,anti_satellite_test_ban,nicaraguan_aid,mx_missile,immigration,synfuels_cutback,education_spending,superfund_lawsuit,crime,duty_free_exports,s_africa_export
party,,,,,,,,,,,,,,,,
republican,0,1,0,1,1,1,0,0,0,1,?,1,1,1,0,1
republican,0,1,0,1,1,1,0,0,0,0,0,1,1,1,0,?
democrat,?,1,1,?,1,1,0,0,0,0,1,0,1,1,0,0
democrat,0,1,1,0,?,1,0,0,0,0,1,0,1,0,0,1
democrat,1,1,1,0,1,1,0,0,0,0,1,?,1,1,1,1


In [37]:
import numpy as np
df = df.replace(np.NaN,0.5)
df.head()

,handicap,water_cost_share,budget_resolution,physician_fee_freeze,el_savador_aid,religious_groups_schools,anti_satellite_test_ban,nicaraguan_aid,mx_missile,immigration,synfuels_cutback,education_spending,superfund_lawsuit,crime,duty_free_exports,s_africa_export
party,,,,,,,,,,,,,,,,
republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0,0.0,1.0
republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.5
democrat,0.5,1.0,1.0,0.5,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
democrat,0.0,1.0,1.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0,1.0


In [0]:
repub = df.loc['republican']
repub.name = 'Republicans'
demo = df.loc['democrat']
demo.name = 'Democrats'

In [59]:
demo.head()

,handicap,water_cost_share,budget_resolution,physician_fee_freeze,el_savador_aid,religious_groups_schools,anti_satellite_test_ban,nicaraguan_aid,mx_missile,immigration,synfuels_cutback,education_spending,superfund_lawsuit,crime,duty_free_exports,s_africa_export
party,,,,,,,,,,,,,,,,
democrat,0.5,1.0,1.0,0.5,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
democrat,0.0,1.0,1.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0,1.0
democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1.0,1.0


In [51]:
a,b = stats.ttest_1samp(demo['handicap'],df['handicap'].mean())
print('%.3f' %(a))

5.339


In [52]:
def function(data, issue):
  a,b =stats.ttest_1samp(data[issue],df[issue].mean())
  print('The T statistic and p value for ',issue,'among ',data.name,'are ',
        '%.3f' %(a),'and ','%.3f' %(b),'respectively.' )
  
function(repub, 'handicap')

The T statistic and p value for  handicap among  Republicans are  -8.306 and  0.000 respectively


In [53]:
for column in repub:
  function(repub, column)

The T statistic and p value for  handicap among  Republicans are  -8.306 and  0.000 respectively
The T statistic and p value for  water_cost_share among  Republicans are  0.069 and  0.945 respectively
The T statistic and p value for  budget_resolution among  Republicans are  -17.091 and  0.000 respectively
The T statistic and p value for  physician_fee_freeze among  Republicans are  57.290 and  0.000 respectively
The T statistic and p value for  el_savador_aid among  Republicans are  25.650 and  0.000 respectively
The T statistic and p value for  religious_groups_schools among  Republicans are  10.821 and  0.000 respectively
The T statistic and p value for  anti_satellite_test_ban among  Republicans are  -9.649 and  0.000 respectively
The T statistic and p value for  nicaraguan_aid among  Republicans are  -14.353 and  0.000 respectively
The T statistic and p value for  mx_missile among  Republicans are  -15.290 and  0.000 respectively
The T statistic and p value for  immigration among 

In [54]:
for column in demo:
  function(demo, column)

The T statistic and p value for  handicap among  Democrats are  5.339 and  0.000 respectively
The T statistic and p value for  water_cost_share among  Democrats are  -0.054 and  0.957 respectively
The T statistic and p value for  budget_resolution among  Democrats are  14.623 and  0.000 respectively
The T statistic and p value for  physician_fee_freeze among  Democrats are  -24.404 and  0.000 respectively
The T statistic and p value for  el_savador_aid among  Democrats are  -11.086 and  0.000 respectively
The T statistic and p value for  religious_groups_schools among  Democrats are  -5.328 and  0.000 respectively
The T statistic and p value for  anti_satellite_test_ban among  Democrats are  7.789 and  0.000 respectively
The T statistic and p value for  nicaraguan_aid among  Democrats are  10.865 and  0.000 respectively
The T statistic and p value for  mx_missile among  Democrats are  9.307 and  0.000 respectively
The T statistic and p value for  immigration among  Democrats are  -1.07

In [58]:
def test(one,two,issue):
  a,b = stats.ttest_ind(one[issue],two[issue])
  print('The T statistic and p value for ',issue,'comparing ',one.name,'to', two.name,'are ',
        a,'and ',b,'respectively.' )
  
test(repub,demo,'handicap')

The T statistic and p value for  handicap comparing  Republicans to Democrats are  -9.22317772154614 and  1.2761169357253626e-18 respectively.


In [60]:
for column in repub:
  test(repub,demo,column)

The T statistic and p value for  handicap comparing  Republicans to Democrats are  -9.22317772154614 and  1.2761169357253626e-18 respectively.
The T statistic and p value for  water_cost_share comparing  Republicans to Democrats are  0.08764559884421878 and  0.9301988772663682 respectively.
The T statistic and p value for  budget_resolution comparing  Republicans to Democrats are  -22.821693043884803 and  2.872115314395808e-76 respectively.
The T statistic and p value for  physician_fee_freeze comparing  Republicans to Democrats are  46.10191006844654 and  3.967141133302638e-169 respectively.
The T statistic and p value for  el_savador_aid comparing  Republicans to Democrats are  20.895617123040896 and  1.4659659186479053e-67 respectively.
The T statistic and p value for  religious_groups_schools comparing  Republicans to Democrats are  9.815876256106362 and  1.142999405504256e-20 respectively.
The T statistic and p value for  anti_satellite_test_ban comparing  Republicans to Democrats

# Resources

- https://homepage.divms.uiowa.edu/~mbognar/applets/t.html
- https://rpsychologist.com/d3/tdist/
- https://gallery.shinyapps.io/tdist/
- https://en.wikipedia.org/wiki/Standard_deviation#Sample_standard_deviation_of_metabolic_rate_of_northern_fulmars